# Based on conversation with ChatGPT : https://chatgpt.com/share/67f4f246-0b88-8012-87c1-8f9023ea0571

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast
from datasets import Dataset
from transformers import BartConfig, BartForConditionalGeneration
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# 1. Generate arithmetic corpus
corpus = [f"{a} + {b} = {a + b}" for a in range(100) for b in range(100)]

# 2. Initialize and train tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
special_tokens = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]
trainer = WordLevelTrainer(special_tokens=special_tokens)
tokenizer.train_from_iterator(corpus, trainer)

# 3. Add post-processing template for special tokens
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]"))
    ]
)

# 4. Save and wrap in HuggingFace tokenizer
tokenizer.save("arithmetic-tokenizer.json")
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="arithmetic-tokenizer.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)
hf_tokenizer.save_pretrained("custom-arithmetic-tokenizer")

# 5. Prepare dataset
examples = [{"input": f"{a} + {b}", "output": str(a + b)} for a in range(100) for b in range(100)]
dataset = Dataset.from_list(examples)

# 6. Tokenize dataset
def tokenize_function(example):
    model_input = hf_tokenizer(example["input"], padding="max_length", truncation=True, max_length=16)
    with hf_tokenizer.as_target_tokenizer():
        labels = hf_tokenizer(example["output"], padding="max_length", truncation=True, max_length=4)
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = dataset.map(tokenize_function, batched=False)

# 7. Define small BART config
config = BartConfig(
    vocab_size=hf_tokenizer.vocab_size,
    d_model=64,
    encoder_layers=1,
    decoder_layers=1,
    encoder_attention_heads=1,
    decoder_attention_heads=1,
    decoder_ffn_dim=32,
    encoder_ffn_dim=32,
    max_position_embeddings=64,
    bos_token_id=hf_tokenizer.bos_token_id,
    eos_token_id=hf_tokenizer.eos_token_id,
    pad_token_id=hf_tokenizer.pad_token_id,
)

# 8. Initialize BART model
model = BartForConditionalGeneration(config)

# 9. Training setup
data_collator = DataCollatorForSeq2Seq(tokenizer=hf_tokenizer, model=model)
training_args = TrainingArguments(
    output_dir="./bart_addition",
    evaluation_strategy="no",
    per_device_train_batch_size=16,
    num_train_epochs=30,
    save_total_limit=1,
    logging_steps=100,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=hf_tokenizer,
    data_collator=data_collator,
)

# 10. Train the model
trainer.train()

2025-04-08 11:48:11.403152: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-08 11:48:11.445784: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 11:48:12.231793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/home/ys/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environ

  0%|          | 0/18750 [00:00<?, ?it/s]

{'loss': 5.0079, 'learning_rate': 4.973333333333334e-05, 'epoch': 0.16}
{'loss': 4.5496, 'learning_rate': 4.9466666666666665e-05, 'epoch': 0.32}
{'loss': 4.1008, 'learning_rate': 4.92e-05, 'epoch': 0.48}
{'loss': 3.6964, 'learning_rate': 4.8933333333333335e-05, 'epoch': 0.64}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


{'loss': 1.139, 'learning_rate': 2.0000000000000003e-06, 'epoch': 28.8}
{'loss': 1.1399, 'learning_rate': 1.7333333333333334e-06, 'epoch': 28.96}
{'loss': 1.1442, 'learning_rate': 1.4666666666666667e-06, 'epoch': 29.12}
{'loss': 1.1393, 'learning_rate': 1.2000000000000002e-06, 'epoch': 29.28}
{'loss': 1.1374, 'learning_rate': 9.333333333333334e-07, 'epoch': 29.44}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


{'loss': 1.1426, 'learning_rate': 6.666666666666667e-07, 'epoch': 29.6}
{'loss': 1.1361, 'learning_rate': 4.0000000000000003e-07, 'epoch': 29.76}
{'loss': 1.1399, 'learning_rate': 1.3333333333333334e-07, 'epoch': 29.92}
{'train_runtime': 449.361, 'train_samples_per_second': 667.615, 'train_steps_per_second': 41.726, 'train_loss': 1.3350471315511068, 'epoch': 30.0}


TrainOutput(global_step=18750, training_loss=1.3350471315511068, metrics={'train_runtime': 449.361, 'train_samples_per_second': 667.615, 'train_steps_per_second': 41.726, 'train_loss': 1.3350471315511068, 'epoch': 30.0})

In [43]:
# 11. Inference
test_inputs = ["12 + 7", "45 + 33", "99 + 1", "0 + 0", "123 + 456"]
inputs = hf_tokenizer(test_inputs, return_tensors="pt", padding=True, truncation=True)

# Generate predictions without token_type_ids
output_ids = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=8)  

# Decode predictions
predictions = hf_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
print(inputs)
print(output_ids)
# 12. Print results
for inp, pred in zip(test_inputs, predictions):
    print(f"{inp} = {pred}")


{'input_ids': tensor([[  2,  93,   4,  98,   3],
        [  2,  60,   4,  72,   3],
        [  2,   6,   4, 104,   3],
        [  2, 105,   4, 105,   3],
        [  2, 129,   4,   1,   3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}
tensor([[  2,   2, 156,   3],
        [  2,   2,   8,   3],
        [  2,   2,   8,   3],
        [  2,   2, 155,   3],
        [  2,   2, 156,   3]])
12 + 7 = 150
45 + 33 = 97
99 + 1 = 97
0 + 0 = 149
123 + 456 = 150


In [47]:
for k,i in hf_tokenizer.vocab.items():
    if i == 2:
        print(k)
        break


[BOS]
